In [62]:
pwd

'/home/akanu/git/deep_sort_yolov3/Bounding_box'

In [63]:
cd '/home/akanu/git/deep_sort_yolov3/Bounding_box'

/home/akanu/git/deep_sort_yolov3/Bounding_box


In [64]:
import pandas as pd
import cv2
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib
import tensorflow.compat.v1 as tf
from tensorflow import keras
import numpy as np
from keras.preprocessing.sequence import pad_sequences
import os


In [66]:
file = "/home/akanu/git/deep_sort_yolov3/Testing_box/04avi.txt"
data = pd.read_csv(file, ' ' )
data
# col = data.columns 
# data.reset_index(inplace=True)
# data.drop('BB_br_1', axis=1, inplace =True)
# data.columns = col
# data.set_index('Frame_Number', inplace =True)

,Frame_Number,Person_ID,State,BB_tl_0,BB_tl_1,BB_br_0,BB_br_1
0,-1,1,1,406,115,450,240
1,-1,2,1,486,116,517,221
2,-1,3,1,270,119,299,220
3,-1,4,1,68,115,118,236
4,-1,5,1,343,120,379,229
...,...,...,...,...,...,...,...
8886,945,92,2,193,118,227,213
8887,945,108,2,60,110,100,227
8888,945,109,2,292,132,321,214
8889,945,110,2,75,97,129,261


In [73]:
data[data['Person_ID'] == 2]['BB_tl_0	BB_tl_1	BB_br_0	BB_br_1'.split()].head()
                    

,BB_tl_0,BB_tl_1,BB_br_0,BB_br_1
1,486,116,517,221
9,487,115,518,220
17,488,114,518,219
26,489,113,519,218
34,490,113,520,219


In [74]:
data[data['Person_ID'] == 2]['BB_tl_0	BB_tl_1	BB_br_0	BB_br_1'.split()].iloc[0:5].values

array([[486, 116, 517, 221],
       [487, 115, 518, 220],
       [488, 114, 518, 219],
       [489, 113, 519, 218],
       [490, 113, 520, 219]])

In [5]:
# I wonder if accuary suffers if I don't sort it
def Files_Load():
    train_file = "/home/akanu/git/deep_sort_yolov3/Bounding_box/"
    test_file = "/home/akanu/git/deep_sort_yolov3/Testing_box/"
    bounding_box_train_txt = os.listdir(train_file)
    bounding_box_train_txt.sort()
    bounding_box_test_txt = os.listdir(test_file)
    bounding_box_test_txt.sort()
    
    loc_files = []
    txt_names = bounding_box_train_txt + bounding_box_test_txt
    
    for txt in bounding_box_train_txt:
        loc_files.append(train_file + txt)
    for txt in bounding_box_test_txt:
        loc_files.append(test_file + txt)
    
    return loc_files, txt_names
    



In [33]:
def Boxes(loc_files, txt_names):
    "File: Points to the folder that has txt files save"

    person_box = []  #Has bounding box locations inside
    frame_person_id = [] #Will have frame Number and person_ID
    video_file = []   #Will Have Video number saved as text file
    split_train_test = 0
    split = 0
    find_split = 0

    for loc, txt_name in zip(loc_files, txt_names):
        data = pd.read_csv(loc, ' ' )
        # Note that person_box is 1 behind ID
        max_person = data['Person_ID'].max()

        for num in range(1,max_person+1):
            person_box.append(data[data['Person_ID'] == num ]['BB_tl_0	BB_tl_1	BB_br_0	BB_br_1'.split()].to_numpy())
            frame_person_id.append(data[data['Person_ID'] == num ]['Frame_Number Person_ID'.split()].to_numpy()) 
            video_file.append(txt_name)
            split_train_test += 1

        del data
        
        find_split += 1
        if txt_name == '16avi.txt' and find_split == 16:
            split = split_train_test
        
        
    return np.array(pad_sequences(person_box, padding='post')), np.array(pad_sequences(frame_person_id, padding='post')), np.array(video_file), split

In [61]:
loc_files, txt_names = Files_Load()
person_box,frame_person_id,video_file, split = Boxes(loc_files, txt_names)

In [35]:
train_person_box, train_frame_person_id, train_video_file =\
    person_box[:split,:,:], frame_person_id[:split,:], video_file[:split]

test_person_box, test_frame_person_id, test_video_file =\
    person_box[split:,:,:], frame_person_id[split:,:], video_file[split:]

In [36]:
train_person_box.shape[-2:]

(1511, 4)

In [37]:
train_frame_person_id.shape

(1327, 1511, 2)

In [38]:
train_video_file.shape

(1327,)

In [39]:
test_person_box.shape

(1406, 1511, 4)

In [40]:
test_frame_person_id.shape

(1406, 1511, 2)

In [52]:
test_video_file.shape

(1406,)

In [60]:
data = pd.read_csv("/home/akanu/git/deep_sort_yolov3/Bounding_box/01avi.txt", ' ')
data[data['Person_ID'] == 1 ]['BB_tl_0	BB_tl_1	BB_br_0	BB_br_1'.split()]

,BB_tl_0,BB_tl_1,BB_br_0,BB_br_1
0,443,107,484,236
7,443,107,484,235
13,444,107,484,234
19,444,106,484,234
25,444,105,485,233
...,...,...,...,...
8256,-32,155,-3,259
8261,-33,155,-4,259
8266,-34,156,-5,260
8271,-35,157,-6,260


In [43]:
data = pd.read_csv("/home/akanu/git/deep_sort_yolov3/Bounding_box/01avi.txt", ' ')
k = data[data['Person_ID'] == 1 ]['BB_tl_0	BB_tl_1	BB_br_0	BB_br_1'.split()].to_numpy()

zero = np.zeros([561,4])
# k =np.concatenate((zero,k), axis=0) #pre
k =np.concatenate((k,zero), axis=0) #post
np.array_equal(k ,person_box[0])
# Shows that when pad is pre this is data taken from first person
# Each batch is a "new person"

True

In [44]:
person_box[0].shape

(1511, 4)

In [45]:
model = keras.Sequential()
model.add(keras.layers.LSTM(4, input_shape= train_person_box.shape[-2:]))
# model = keras.Sequential([

#     keras.layers.Flatten(input_shape=(28, 28)),
#     keras.layers.Dense(128, activation=tf.nn.relu),
#     keras.layers.Dense(10, activation=tf.nn.softmax)
# ])
model.compile(optimizer='adam', loss='mae')


In [24]:
model.fit(x=train_person_box, epochs=3)

Epoch 1/3
1327/1327 [==============================] - 81s 61ms/sample - loss: 0.6828
Epoch 2/3
1327/1327 [==============================] - 80s 60ms/sample - loss: 0.5912
Epoch 3/3
1327/1327 [==============================] - 80s 60ms/sample - loss: 0.5534


In [27]:
model.predict(test_person_box[0:32])

array([[ 0.7615942 ,  0.        ,  0.7615942 , -0.        ],
       [-0.99999905, -0.        ,  0.7615942 ,  1.        ],
       [ 0.7615942 ,  0.        ,  0.7615942 , -0.        ],
       [ 0.7615942 ,  0.        ,  0.7615942 , -0.        ],
       [-1.        , -0.        ,  0.7615942 ,  1.        ],
       [ 0.7615942 ,  0.        ,  0.7615942 , -0.        ],
       [ 0.7615942 ,  0.        ,  0.7615942 , -0.        ],
       [ 0.7615942 , -0.        ,  0.7615942 ,  0.9894832 ],
       [ 0.7615942 ,  0.        ,  0.7615942 , -0.        ],
       [ 0.7615942 ,  0.        ,  0.7615942 , -0.        ],
       [ 0.7615942 ,  0.        ,  0.7615942 ,  0.92481774],
       [ 0.7615942 ,  0.        ,  0.7615942 ,  0.        ],
       [ 0.7615942 , -0.        ,  0.7615942 ,  0.92481774],
       [ 0.7615942 ,  0.        ,  0.7615942 , -0.        ],
       [ 0.7615942 ,  0.        ,  0.7615942 , -0.        ],
       [ 0.7615942 ,  0.        ,  0.        , -0.        ],
       [ 0.7615942 ,  0.

In [46]:
model.fit(x=train_person_box, epochs=3)

Epoch 1/3
1327/1327 [==============================] - 81s 61ms/sample - loss: 0.9825
Epoch 2/3
1327/1327 [==============================] - 80s 60ms/sample - loss: 0.9151
Epoch 3/3
1327/1327 [==============================] - 80s 60ms/sample - loss: 0.5925


# Want to know if putting zeros in front or back effects training loss

In [51]:
model.predict(test_person_box[0:32])

array([[0.54451126, 0.6337535 , 0.57021266, 0.68583274],
       [0.54451126, 0.6337535 , 0.57021266, 0.68583274],
       [0.54451126, 0.6337535 , 0.57021266, 0.68583274],
       [0.54451126, 0.6337535 , 0.57021266, 0.68583274],
       [0.54451126, 0.6337535 , 0.57021266, 0.68583274],
       [0.54451126, 0.6337535 , 0.57021266, 0.68583274],
       [0.54451126, 0.6337535 , 0.57021266, 0.68583274],
       [0.54451126, 0.6337535 , 0.57021266, 0.68583274],
       [0.54451126, 0.6337535 , 0.57021266, 0.68583274],
       [0.54451126, 0.6337535 , 0.57021266, 0.68583274],
       [0.54451126, 0.6337535 , 0.57021266, 0.68583274],
       [0.54451126, 0.6337535 , 0.57021266, 0.68583274],
       [0.54451126, 0.6337535 , 0.57021266, 0.68583274],
       [0.54451126, 0.6337535 , 0.57021266, 0.68583274],
       [0.54451126, 0.6337535 , 0.57021266, 0.68583274],
       [0.54451126, 0.6337535 , 0.57021266, 0.68583274],
       [0.54451126, 0.6337535 , 0.57021266, 0.68583274],
       [0.54451126, 0.6337535 ,

In [ ]:
BATCH_SIZE = 256
BUFFER_SIZE = 500

train_univariate = tf.data.Dataset.from_tensor_slices((train_person_box))
train_univariate = train_univariate.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_univariate = tf.data.Dataset.from_tensor_slices((test_person_box))
val_univariate = val_univariate.batch(BATCH_SIZE).repeat()

In [ ]:
model1 = keras.Sequential()
model1.add(keras.layers.LSTM(15, input_shape= train_person_box.shape[-2:]))
# model = keras.Sequential([

#     keras.layers.Flatten(input_shape=(28, 28)),
#     keras.layers.Dense(128, activation=tf.nn.relu),
#     keras.layers.Dense(10, activation=tf.nn.softmax)
# ])
model1.compile(optimizer='adam', loss='mae')
